In [7]:
import rltk

In [63]:
class Record1(rltk.Record):
   @property
   def id(self):
      return self.raw_object['histid']

   @property
   def value(self):
      return self.raw_object['namelast']
   
   def firstname(self):
      return self.raw_object['namefrst']
   
   def age(self):
      return self.raw_object['age']
   
   def birthplace(self):
      return self.raw_object['bpl']
   


class Record2(rltk.Record):
   @rltk.cached_property
   def id(self):
      return self.raw_object['histid']

   @rltk.cached_property
   def value(self):
      return self.raw_object['namelast']
   
   def firstname(self):
      return self.raw_object['namefrst']

   def age(self):
      return self.raw_object['age'] 

   def birthplace(self):
      return self.raw_object['bpl']  

      # v = self.raw_object.get('values', list())
      # return v[0] if len(v) > 0 else 'empty'

In [64]:
ds1 = rltk.Dataset(reader=rltk.CSVReader('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1880 from 1880-1900 experiment.csv'),
                     record_class=Record1, adapter=rltk.MemoryKeyValueAdapter())

In [65]:
ds2 = rltk.Dataset(reader=rltk.CSVReader('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1900 from 1880-1900 experiment.csv'),
                     record_class=Record2, adapter=rltk.MemoryKeyValueAdapter())

In [72]:
pairs = rltk.get_record_pairs(ds1, ds2)

In [73]:
iteration = 0
for r1, r2 in pairs:
    iteration += 1
    print('-------------')
    print(r1.id, r1.firstname(), r2.id, r2.firstname())
    print('levenshtein_distance:', rltk.levenshtein_distance(r1.firstname(), r2.firstname()))
    print('levenshtein_similarity:', rltk.levenshtein_similarity(r1.firstname(), r2.firstname()))


-------------
d61c52ce-9753-4559-9235-b54ebdc44547 HENRY 2F10B1BD-E259-4843-8B16-5F9E7B67B4AD JONAS S
levenshtein_distance: 6
levenshtein_similarity: 0.1428571428571429
-------------
d61c52ce-9753-4559-9235-b54ebdc44547 HENRY 7F1FAACE-1662-4653-AB78-78C905E094C2 BOB
levenshtein_distance: 5
levenshtein_similarity: 0.0
-------------
d61c52ce-9753-4559-9235-b54ebdc44547 HENRY 9B918B62-1FA5-41F9-B517-76A85667BD41 NED
levenshtein_distance: 4
levenshtein_similarity: 0.19999999999999996
-------------
161e51f4-f31f-4a8a-9a54-5f1627acefef CHARLES J. 2F10B1BD-E259-4843-8B16-5F9E7B67B4AD JONAS S
levenshtein_distance: 8
levenshtein_similarity: 0.19999999999999996
-------------
161e51f4-f31f-4a8a-9a54-5f1627acefef CHARLES J. 7F1FAACE-1662-4653-AB78-78C905E094C2 BOB
levenshtein_distance: 10
levenshtein_similarity: 0.0
-------------
161e51f4-f31f-4a8a-9a54-5f1627acefef CHARLES J. 9B918B62-1FA5-41F9-B517-76A85667BD41 NED
levenshtein_distance: 9
levenshtein_similarity: 0.09999999999999998
-------------

In [67]:
for r1, r2 in pairs:
    print(r1.birthplace())
    print(type(r1.birthplace()))

Canada
<class 'str'>
Canada
<class 'str'>
Canada
<class 'str'>
Missouri
<class 'str'>
Missouri
<class 'str'>
Missouri
<class 'str'>
Canada
<class 'str'>
Canada
<class 'str'>
Canada
<class 'str'>
Canada
<class 'str'>
Canada
<class 'str'>
Canada
<class 'str'>
Argentina
<class 'str'>
Argentina
<class 'str'>
Argentina
<class 'str'>


In [1]:
import pandas as pd

ds1 = pd.read_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1880 from 1880-1900 experiment.csv')
ds2 = pd.read_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1900 from 1880-1900 experiment.csv')

In [2]:
ds1["matching_id"] = None
ds2["matching_id"] = None

In [4]:
for index1, row1 in ds1.iterrows():
    print(index1)

0
1
2
3
4


In [8]:
def create_dataframe(dataset1, dataset2):

    df = {index1: [] for index1, row1 in dataset1.iterrows()}
    

    for index1, row1 in dataset1.iterrows():
        score_list = []
        
        
        histid1 = row1['histid']
        firstname1 = row1['namefrst']
        lastname1 = row1['namelast']
        birthplace1 = row1['bpl']

        for index2, row2 in dataset2.iterrows():

            histid2 = row2['histid']
            firstname2 = row2['namefrst']
            lastname2 = row2['namelast']
            birthplace2 = row2['bpl']  

            histid_score =  rltk.levenshtein_similarity(histid1, histid2)*0.4
            firstname_score = rltk.levenshtein_similarity(firstname1, firstname2)*0.2
            lastname_score = rltk.levenshtein_similarity(lastname1, lastname2)*0.2

            if birthplace1 == birthplace2:
                birthplace_score = 1*0.2
            else:
                birthplace_score = 0*0.2

            total_score = histid_score + firstname_score + lastname_score + birthplace_score
            
            score_list.append(total_score)

        df[index1].extend(score_list)

    return pd.DataFrame(df)


In [15]:
score_df = create_dataframe(ds1, ds2)
create_dataframe(ds1, ds2)


,0,1,2,3,4
0,0.117460,0.168889,0.095238,0.055556,0.125000
1,0.077778,0.066667,0.197778,0.077778,0.111111
2,0.184444,0.120000,0.188889,0.077778,0.158333


In [11]:
# dictionary to control to which record in ds2 each record in ds1 can match
ds2_record_available = {index: [i for i in range(ds2.shape[0])] for index, row in ds1.iterrows()}

# waiting list
waiting_list = []

# dictionary to store created pairs
pairs = {}

# variables to count the number of iterations
count = 0

In [13]:
ds2_record_available

{0: [0, 1, 2], 1: [0, 1, 2], 2: [0, 1, 2], 3: [0, 1, 2], 4: [0, 1, 2]}

In [14]:
len(ds1)

5

In [ ]:
# while not all record in ds1 has matches
while len(waiting_list) < len(ds1):

    # look for matchs
    for index1 in range(len(ds1)):

        if index1 not in waiting_list:
            # each ds1 record tries to find match in ds2 with the highest score
            index2_available = ds2_record_available[index1]
            best_choice = score_df

In [18]:
score_df.iloc[0, 0]

0.11746031746031747

In [ ]:
def Gale_Shapley():
    

In [19]:
# Dictionary to store unique matching IDs
matching_ids = {}
next_id = 1

# Iterate through the lists to identify matching pairs and assign matching IDs
for idxA, idxB in zip(l1, l2):
    # Check if matching IDs are already assigned
    if ds1.loc[idxA, 'matching_id'] is not None:
        matching_id = ds1.loc[idxA, 'matching_id']
    elif ds2.loc[idxB, 'matching_id'] is not None:
        matching_id = ds2.loc[idxB, 'matching_id']
    else:
        # Generate a new matching ID
        matching_id = next_id
        matching_ids[(idxA, idxB)] = matching_id
        next_id += 1
    
    # Update matching ID for dataframe 1
    ds1.at[idxA, 'matching_id'] = matching_id
    
    # Update matching ID for dataframe 2
    ds2.at[idxB, 'matching_id'] = matching_id

In [20]:
ds1

,year,serial,stateicp,city,countynhg,yrstcounty,stcounty,countyicp,famsize,eldch,...,sex,race,bpl,occ1950,namelast,namefrst,bplstr,histid,pid,matching_id
0,1880,383439,Arkansas,BIG ROCK,501190,1880421190,51190,1190,6 family members present,15.0,...,Male,Mulatto,Canada,Carpenters,WALLACE,HENRY,CANADA,d61c52ce-9753-4559-9235-b54ebdc44547,01254054-2-0264-0806-$,1
1,1880,426308,California,OAKLAND,600010,1880710010,60010,10,8 family members present,9.0,...,Male,Mulatto,Missouri,"Painters, construction and maintenance",KYER,CHARLES J.,MO,161e51f4-f31f-4a8a-9a54-5f1627acefef,01254061-1-0136-1756-$,2
2,1880,475226,California,SAN RAFAEL,600410,1880710410,60410,410,3 family members present,NaN,...,Male,Black/African American/Negro,Canada,Taxicab drivers and chauffers,BROM,DANIEL,CAN,b3f9906c-e396-4435-90c0-72e9c1f43177,01254068-1-0080-0959-$,3
3,1880,475252,California,SAN RAFAEL,600410,1880710410,60410,410,7 family members present,14.0,...,Male,Black/African American/Negro,Canada,"Farm laborers, wage workers",TALBOT,ARTHUR,CAN,7ee3ec94-fb88-4978-b7d5-8150e0163f0a,01254068-1-0081-1092-$,3
4,1880,519735,California,SAN FRANCISCO,600750,1880710750,60750,750,4 family members present,18.0,...,Female,White,Argentina,Keeps house/housekeeping at home/housewife,BAYLEY,EMILE P.,BUENOS AYRES,7899c3f7-c939-4cfe-9734-39e3951469c5,01254073-1-0246-4060-$,1


In [21]:
ds2

,year,serial,stateicp,city,stcounty,countyicp,famsize,eldch,yngch,age,sex,race,bpl,occ1950,namelast,namefrst,bplstr,histid,pid,matching_id
0,1900,110663,Alabama,"PRECINCT 36, SELMA",10470,470,4 family members present,NaN,NaN,30,Male,Black/African American/Negro,Texas,Teachers (n.e.c.),MOTEN,JONAS S,TEXAS,2F10B1BD-E259-4843-8B16-5F9E7B67B4AD,4119981_00297,2
1,1900,140747,Alabama,"PRECINCT 3, TISHABEE",10630,630,2 family members present,NaN,NaN,45,Male,Black/African American/Negro,Alabama,Farmers (owners and tenants),BROWN,BOB,ALABAMA,7F1FAACE-1662-4653-AB78-78C905E094C2,4119984_00570,3
2,1900,223762,Alabama,"PRECINCT 10, FORT DEPOSIT",10850,850,12 family members present,19.0,3.0,49,Male,Black/African American/Negro,Alabama,Farmers (owners and tenants),BARGANIER,NED,ALABAMA,9B918B62-1FA5-41F9-B517-76A85667BD41,4119997_00022,1


In [95]:
for index, row in ds1.iterrows():
    print(index)
    print(row['namefrst'], row['namelast'])

0
HENRY WALLACE
1
CHARLES J. KYER
2
DANIEL BROM
3
ARTHUR TALBOT
4
EMILE P. BAYLEY


In [16]:
ds1.loc[0, 'matching_id'] == None

True

In [17]:
ds1.shape

(5, 22)

In [18]:
ds2.shape

(3, 20)